<div style="direction:rtl;line-height:300%;"> <font face="XB Zar">
<div align=center>
    <div style="font-size:14px">
    به نام خدا
    </div><br>
    <strong><div style="font-size:50px; color: blue">
    ci - hw1
    </div></strong>
    <br>
    <div style="font-size:20px; color: #ff6600">
    مائده نادهی 9712762238
    </div>
</div>
<hr>

<div style="direction:rtl; font-size: 17px; font-family: 'XB Zar'; line-height: 200%">
<div style="font-size: 22px; color: #ff6600; font-weight: bold">
فهرست محتویات
</div>         
<ul style="margin-right: 0;">
    <li><a href="#section1">preprocessing</a></li>
    <li><a href="#section2">Simple K-means</a></li>
    <li><a href="#section3">K-Means Optimisation</a></li>
    <li><a href="#section4">Mean-Shift</a></li>
    <li><a href="#section5">DBSCAN</a></li>
    <li><a href="#section6">Agglomerative</a></li>
</ul>
</div>

In [2]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import seaborn as sns
import math
from sklearn import metrics
from sklearn.cluster import OPTICS
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import MeanShift, estimate_bandwidth
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unzip '/content/drive/MyDrive/MNIST dataset - train.zip'

Archive:  /content/drive/MyDrive/MNIST dataset - train.zip
  inflating: train_label.csv         
  inflating: train_set.csv           


In [3]:
train_label = pd.read_csv("/content/train_label.csv")
train_label.rename(columns = {"5" : "actual_labels"}, inplace=True)
train_label = train_label["actual_labels"]
train_set = pd.read_csv("/content/train_set.csv")
train_set.set_axis([i for i in range(0,784)], axis=1, inplace=True)

<div id="section1" style="direction:rtl; font-size: 17px; font-family: 'XB Zar'; line-height: 200%">
<div style="font-size: 22px; color: #ff6600; font-weight: bold">
preprocessing
</div>
<hr>
<div>
ابتدا بررسی میکنیم داده ی غیر معتبر NULL و NAN نداشته باشیم :
</div>
</div>

In [4]:
# how many null?
train_set.isnull().sum().isnull().sum(axis = 0)

0

<div style="direction:rtl; font-size: 17px; font-family: 'XB Zar'; line-height: 200%">
<div>
تعداد زیادی(67) ستون تماما صفر داریم که این فیچر ها چون در همه ی data point مقدار برابر صفر دارند باعث ایجاد وجه تمایز بین data point ها نمیشود و میتوان آن ها را حذف نمود :
</div>
</div>

In [4]:
X_282features = train_set.loc[:, train_set.std() > 0.3]

# remove all zero columns
zero_columns = []
for i in range(0, len(train_set.columns)):
    if train_set[i].sum() == 0:
        zero_columns.append(i)
print('number of zero columns : ', len(zero_columns))
train_set.drop(train_set.columns[zero_columns], axis=1, inplace=True) 

number of zero columns :  67


<div style="direction:rtl; font-size: 17px; font-family: 'XB Zar'; line-height: 200%">
<div>
   از بخش کوچکی از دیتا (یک شصتم) استفاده میکنیم که با استفاده از آن مقادیر مناسب هایپرپارامترها را حدس بزنیم همچنین برای مقایسه و انتخاب متد بهتر از این ساب ست استفاده میکنیم :
</div>
</div>

In [5]:
X = train_set[:1000]
y_true = train_label[:1000]

In [6]:
def eval_metrics(description, n_clusters, y_true, y_pred):
  print(description)
  print('Number of clusters: {}'.format(n_clusters))
  print('Accuracy: {:.2f}'.format(metrics.accuracy_score(y_true, y_pred)))
  print('Rand Index: {:.2f}'.format(metrics.rand_score(y_true, y_pred)))
  print('------------------------')

def get_accuracy(y_true, y_pred):
  return metrics.accuracy_score(y_true, y_pred)

In [7]:
def map_cluster(y_clusters, y, numClusters):
    """
    Parameters:
        y, a 1-D Numpy array containing true class numbers (integers between 0 and 10)
        y_cluster, a 1-D Numpy array containing cluster numbers corresponding to each element of y
        
    Returns:
        y_pred, a 1-D Numpy array mapping clusters to most common class in y 
    """
    y_pred = np.zeros(y_clusters.shape).astype(np.int32)
    mapOfCluster = np.zeros(numClusters).astype(np.int32) * -1
    # Map clusters to digits in mapOfClusters by choosing the digit most often associated with each cluster
    for cluster in range(0, numClusters):
        # Count number of true labels for each member in this cluster
        count = np.bincount(y[y_clusters == cluster], minlength=10)
        # Pick the index of the first max valuein count if there are more than max values
        mapOfCluster[cluster] = np.argmax(count)

    y_pred = mapOfCluster[y_clusters]
    return y_pred

In [8]:
def get_n_clusters(model):
  labels = model.labels_
  return len(np.unique(labels))

In [9]:
k = len(np.unique(train_label))
print('k : ', k)

def k_means(X, y_true, n_clusters):
  Kclustering = KMeans(n_clusters=n_clusters, random_state=0)
  t1 = time.time()
  Kclustering.fit(X)
  t2 = time.time()
  print('time : ', t2 - t1)
  relabel = map_cluster(Kclustering.labels_, y_true, n_clusters)
  eval_metrics("k-means clustering", n_clusters, y_true, relabel)

def miniBatchKMeans(X, y_true, n_clusters):
  MBClustering = MiniBatchKMeans(n_clusters=n_clusters, random_state=0, batch_size=6)
  t1 = time.time()
  MBClustering.fit(X)
  t2 = time.time()
  print('time : ', t2 - t1)
  relabel = map_cluster(MBClustering.labels_, y_true, n_clusters)
  eval_metrics("mini batch k-means clustering", n_clusters, y_true, relabel)

k :  10


In [10]:
def mean_shift(X, y_true, quantile):
  bandwidth = estimate_bandwidth(X, quantile=quantile, n_samples=(len(X)//3))
  # MClustering = MeanShift(bandwidth=bandwidth, bin_seeding=True)
  MClustering = MeanShift(bandwidth=bandwidth)
  t1 = time.time()
  MClustering.fit(X)
  t2 = time.time()
  print('time : ', t2 - t1)
  n = get_n_clusters(MClustering)
  relabel = map_cluster(MClustering.labels_, y_true, n)
  eval_metrics("mean-shift clustering - quantile:" + str(quantile) + " - bandwidth:" + str(bandwidth), n, y_true, relabel)

In [11]:
def dbscan(X, y_true, epsilon, min_samples, print_it:bool):
  DClustering = DBSCAN(eps=epsilon, min_samples=min_samples, n_jobs=-1)
  t1 = time.time()
  DClustering.fit(X)
  t2 = time.time()
  n = get_n_clusters(DClustering)
  relabel = map_cluster(DClustering.labels_, y_true, n)
  if print_it or get_accuracy(y_true, relabel) > 0.4:
    print('time : ', t2 - t1)
    eval_metrics("DBSCAN clustering - epsilon:" + str(epsilon) + " - min_samples:" + str(min_samples), n, y_true, relabel)

In [12]:
def do_agglo_clustering(X, y_true, n_clusters, affinity, linkage):
  Hclustering = AgglomerativeClustering(n_clusters=n_clusters, affinity=affinity, linkage=linkage)
  t1 = time.time()
  Hclustering.fit(X)
  t2 = time.time()
  #WE NEED TO MAP THE CLUSTERS TO THEIR RESPECTIVE TRUTH LABELS HERE (cluster, maplabel)
  relabel = map_cluster(Hclustering.labels_, y_true, n_clusters)
  print('time : ', t2 - t1)
  eval_metrics("agglomerative clustering - " + affinity + " - " + linkage, n_clusters, y_true, relabel)

def agglomerative(X, y_true, n_clusters):
  do_agglo_clustering(X, y_true, n_clusters,'euclidean', 'ward')
  do_agglo_clustering(X, y_true, n_clusters,'euclidean', 'complete')
  do_agglo_clustering(X, y_true, n_clusters,'euclidean', 'average')
  do_agglo_clustering(X, y_true, n_clusters,'manhattan', 'average')
  do_agglo_clustering(X, y_true, n_clusters,'cosine', 'average')

<div style="direction:rtl; font-size: 17px; font-family: 'XB Zar'; line-height: 200%">
<div>
    از pca برای کاهش بعد استفاده میکنیم (چهت سرعت محاسبات بالاتر مورد مقایسه قرار میگیرد اگر باعث کاهش purity نشود از دیتاست با ابعاد کوچک تر استفاده میکنیم چون زمان اجرای الگوریتم ها روی این دیتاست کمتر خواهد بود)
</div>
</div>

In [13]:
def do_pca(n_components, data):
    X = StandardScaler().fit_transform(data)
    pca = PCA(n_components=n_components)
    X_pca = pca.fit_transform(X)
    # X_pca = StandardScaler().fit_transform(X_pca)
    return pca, X_pca

pca, X_pca = do_pca(59, X)   
print('number of features : ', pca.n_components_) 

number of features :  59


In [14]:
pca2, train_set_pca = do_pca(59, train_set)

In [40]:
!pip install -U scikit-learn

     |████████████████████████████████| 23.2 MB 1.3 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


<div id="section2" style="direction:rtl; font-size: 17px; font-family: 'XB Zar'; line-height: 200%">
<div style="font-size: 22px; color: #ff6600; font-weight: bold">
Simple K-means
</div>
<hr>
</div>

In [32]:
# without pca
k_means(X, y_true, k)
miniBatchKMeans(X, y_true, k)

# with pca
k_means(X_pca, y_true, k)
miniBatchKMeans(X_pca, y_true, k)

# with df_282features

time :  1.4952552318572998
k-means clustering
Number of clusters: 10
Accuracy: 0.54
Rand Index: 0.85
------------------------
time :  0.2023000717163086
mini batch k-means clustering
Number of clusters: 10
Accuracy: 0.58
Rand Index: 0.85
------------------------
time :  0.918381929397583
k-means clustering
Number of clusters: 10
Accuracy: 0.46
Rand Index: 0.84
------------------------
time :  0.01722884178161621
mini batch k-means clustering
Number of clusters: 10
Accuracy: 0.49
Rand Index: 0.85
------------------------


<div style="direction:rtl; font-size: 17px; font-family: 'XB Zar'; line-height: 200%">
<div>
    در نتیجه بهترین انتخاب : 
</div>
</div>

In [230]:
# => without pca (better accuracy) & minibatch (faster)
miniBatchKMeans(train_set, train_label, k)

time :  2.5096120834350586
mini batch k-means clustering
Number of clusters: 10
Accuracy: 0.56
Rand Index: 0.85
------------------------


<div id="section3" style="direction:rtl; font-size: 17px; font-family: 'XB Zar'; line-height: 200%">
<div style="font-size: 22px; color: #ff6600; font-weight: bold">
K-Means Optimisation
</div>
<hr>
<div>
برای بهبود، سعی می کنیم عملکرد را برای مقادیر مختلف k> 10 ببینیم. به این ترتیب، ارقام یک برچسب اما با ویژگی های متفاوت ممکن است بهتر خوشه بندی شوند. اگر اجازه "split" داده شود.

</div>
<div>
    accuracy محاسبه می‌شود که مشاهده می‌کنیم که با مقدار k افزایش می‌یابد.
    <span style="color: red">مشاهده می شود که به accuracy نود و سه (93) درصد هم میرسد</span>
</div>
</div>

In [44]:
miniBatchKMeans(train_set, train_label, 10)
miniBatchKMeans(train_set, train_label, 16)
miniBatchKMeans(train_set, train_label, 32)
miniBatchKMeans(train_set, train_label, 128)
miniBatchKMeans(train_set, train_label, 256)

time :  2.5496342182159424
mini batch k-means clustering
Number of clusters: 10
Accuracy: 0.56
Rand Index: 0.85
------------------------
time :  2.0078980922698975
mini batch k-means clustering
Number of clusters: 16
Accuracy: 0.66
Rand Index: 0.90
------------------------
time :  2.795400381088257
mini batch k-means clustering
Number of clusters: 32
Accuracy: 0.72
Rand Index: 0.92
------------------------
time :  0.9784281253814697
mini batch k-means clustering
Number of clusters: 128
Accuracy: 0.73
Rand Index: 0.91
------------------------
time :  2.341218948364258
mini batch k-means clustering
Number of clusters: 256
Accuracy: 0.80
Rand Index: 0.93
------------------------


<div id="section6" style="direction:rtl; font-size: 17px; font-family: 'XB Zar'; line-height: 200%">
<div style="font-size: 22px; color: #ff6600; font-weight: bold">
Agglomerative
</div>
<hr>
</div>

In [35]:
# without pca
agglomerative(X, y_true, 10)

time :  0.31000375747680664
agglomerative clustering - euclidean - ward
Number of clusters: 10
Accuracy: 0.57
Rand Index: 0.85
------------------------
time :  0.2957947254180908
agglomerative clustering - euclidean - complete
Number of clusters: 10
Accuracy: 0.42
Rand Index: 0.77
------------------------
time :  0.29579925537109375
agglomerative clustering - euclidean - average
Number of clusters: 10
Accuracy: 0.29
Rand Index: 0.42
------------------------
time :  0.3062312602996826
agglomerative clustering - manhattan - average
Number of clusters: 10
Accuracy: 0.29
Rand Index: 0.42
------------------------
time :  0.2979471683502197
agglomerative clustering - cosine - average
Number of clusters: 10
Accuracy: 0.27
Rand Index: 0.51
------------------------


In [36]:
# with pca
agglomerative(X_pca, y_true, 10)

time :  0.04931163787841797
agglomerative clustering - euclidean - ward
Number of clusters: 10
Accuracy: 0.48
Rand Index: 0.84
------------------------
time :  0.03660011291503906
agglomerative clustering - euclidean - complete
Number of clusters: 10
Accuracy: 0.13
Rand Index: 0.13
------------------------
time :  0.03689980506896973
agglomerative clustering - euclidean - average
Number of clusters: 10
Accuracy: 0.13
Rand Index: 0.12
------------------------
time :  0.03644227981567383
agglomerative clustering - manhattan - average
Number of clusters: 10
Accuracy: 0.13
Rand Index: 0.12
------------------------
time :  0.03510880470275879
agglomerative clustering - cosine - average
Number of clusters: 10
Accuracy: 0.49
Rand Index: 0.85
------------------------


In [15]:
# => euclidean - ward & without pca (better accuracy)
do_agglo_clustering(train_set[:20000], train_label[:20000], 10, 'euclidean', 'ward')
# => cosine - average & with pca (faster)
do_agglo_clustering(train_set_pca[:20000], train_label[:20000], 10, 'cosine', 'average')

time :  158.0088987350464
agglomerative clustering - euclidean - ward
Number of clusters: 10
Accuracy: 0.61
Rand Index: 0.90
------------------------
time :  24.031623363494873
agglomerative clustering - cosine - average
Number of clusters: 10
Accuracy: 0.59
Rand Index: 0.88
------------------------


<div id="section4" style="direction:rtl; font-size: 17px; font-family: 'XB Zar'; line-height: 200%">
<div style="font-size: 22px; color: #ff6600; font-weight: bold">
Mean-Shift
</div>
<hr>
</div>

In [66]:
# without pca
mean_shift(X, y_true, 0.2)
mean_shift(X, y_true, 0.5)
mean_shift(X, y_true, 0.7)

time :  26.036401510238647
mean-shift clustering - quantile:0.2 - bandwidth:9.142767959197903
Number of clusters: 1
Accuracy: 0.12
Rand Index: 0.10
------------------------
time :  22.71328568458557
mean-shift clustering - quantile:0.5 - bandwidth:10.065149020488343
Number of clusters: 1
Accuracy: 0.12
Rand Index: 0.10
------------------------
time :  18.53302526473999
mean-shift clustering - quantile:0.7 - bandwidth:10.634405566827192
Number of clusters: 1
Accuracy: 0.12
Rand Index: 0.10
------------------------


In [78]:
# with pca
mean_shift(X_pca, y_true, 0.2)
mean_shift(X_pca, y_true, 0.5)
mean_shift(X_pca, y_true, 0.7)

time :  4.6385791301727295
mean-shift clustering - quantile:0.2 - bandwidth:22.70615367230931
Number of clusters: 48
Accuracy: 0.20
Rand Index: 0.24
------------------------
time :  4.538547039031982
mean-shift clustering - quantile:0.5 - bandwidth:25.84077273399982
Number of clusters: 32
Accuracy: 0.16
Rand Index: 0.17
------------------------
time :  4.626535177230835
mean-shift clustering - quantile:0.7 - bandwidth:28.135176219133985
Number of clusters: 29
Accuracy: 0.15
Rand Index: 0.16
------------------------


In [80]:
pca2, X_pca_2 = do_pca(2, X) #with 2 dimension

In [81]:
# with pca2
mean_shift(X_pca_2, y_true, 0.1)
mean_shift(X_pca_2, y_true, 0.3)

time :  9.484244585037231
mean-shift clustering - quantile:0.1 - bandwidth:3.996780170203871
Number of clusters: 10
Accuracy: 0.24
Rand Index: 0.62
------------------------
time :  5.644194602966309
mean-shift clustering - quantile:0.3 - bandwidth:6.998810091228253
Number of clusters: 2
Accuracy: 0.16
Rand Index: 0.23
------------------------


In [31]:
# => with pca (accuracy and faster)
mean_shift(train_set_pca[:20000], train_label[:20000], 0.2)

time :  618.7734246253967
mean-shift clustering - quantile:0.2 - bandwidth:22.63214451701725
Number of clusters: 152
Accuracy: 0.15
Rand Index: 0.18
------------------------


In [45]:
mean_shift(train_set_pca[:10000], train_label[:10000], 0.5)
mean_shift(train_set_pca[:10000], train_label[:10000], 0.1)

time :  157.19813656806946
mean-shift clustering - quantile:0.5 - bandwidth:25.459846756762172
Number of clusters: 62
Accuracy: 0.14
Rand Index: 0.15
------------------------
time :  177.41390895843506
mean-shift clustering - quantile:0.1 - bandwidth:20.66144011450685
Number of clusters: 152
Accuracy: 0.18
Rand Index: 0.24
------------------------


<div id="section5" style="direction:rtl; font-size: 17px; font-family: 'XB Zar'; line-height: 200%">
<div style="font-size: 22px; color: #ff6600; font-weight: bold">
DBSCAN
</div>
<hr>
</div>

In [223]:
# without pca
bandwidth = estimate_bandwidth(X, n_samples=(len(X)//3))
bandwidth = math.floor(bandwidth)
print('bandwidth: ', bandwidth)
for eps in range(1, 2*bandwidth):
  for minSample in range(2, (bandwidth)):
    dbscan(X, y_true, epsilon=eps, min_samples=minSample, print_it=False)

bandwidth:  9
time :  0.12993717193603516
DBSCAN clustering - epsilon:5 - min_samples:2
Number of clusters: 65
Accuracy: 0.47
Rand Index: 0.66
------------------------
time :  0.13017988204956055
DBSCAN clustering - epsilon:5 - min_samples:3
Number of clusters: 25
Accuracy: 0.41
Rand Index: 0.60
------------------------


In [228]:
# with pca 
dbscan(X_pca, y_true, epsilon=5, min_samples=2, print_it=True)
dbscan(X_pca, y_true, epsilon=10, min_samples=2, print_it=True)

time :  0.11869621276855469
DBSCAN clustering - epsilon:5 - min_samples:2
Number of clusters: 14
Accuracy: 0.21
Rand Index: 0.29
------------------------
time :  0.11542320251464844
DBSCAN clustering - epsilon:10 - min_samples:2
Number of clusters: 39
Accuracy: 0.32
Rand Index: 0.61
------------------------


In [33]:
print(estimate_bandwidth(X, n_samples=(len(X)//3)))
print(estimate_bandwidth(train_set[:20000], n_samples=(len(X)//3)))

9.500968533427843
9.66163965427474


In [42]:
# => without pca (better accuracy)
dbscan(train_set[:20000], train_label[:20000], epsilon=5, min_samples=3, print_it=True)

time :  27.847715616226196
DBSCAN clustering - epsilon:5 - min_samples:3
Number of clusters: 153
Accuracy: 0.18
Rand Index: 0.53
------------------------
